Using MCMC on simulated S82 LCs, as well as MAP.  

Storing both the full chains, MAP estimates, and logL evaluated on  a grid . 

Do all for Jeff2 , same prior as used by Kozlowski (if needed, can repeat with the other prior too )

In [ ]:
import numpy as np 
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib import rcParams 
from astroML.plotting.mcmc import convert_to_stdev
from astroML.datasets import fetch_dr7_quasar
import celerite
from celerite import terms
from astropy.table import Table
from astropy.table import vstack
from astropy.table import Column
import emcee
rcParams['ytick.labelsize'] = 15
rcParams['xtick.labelsize'] = 15
rcParams['axes.labelsize'] = 20
rcParams['axes.linewidth'] = 2
rcParams['font.size'] = 15
rcParams['axes.titlesize'] = 18

Simulate SDSS light curves over 8 years , 60 points 

In [ ]:
t_exp = 8 * 365 # days 
rho_in = np.array([0.001, 15])
tau_in = rho_in * t_exp
print(' %.2f < tau_in < %.2f  [days]'% (tau_in[0], tau_in[1]))

In [ ]:
# define the negative log likelihood
def neg_log_posterior(params,y,gp,prior):
    if prior is 'None' : 
        gp.set_parameter_vector(params)
        return -gp.log_likelihood(y, quiet=True)

    if prior is 'Jeff1' : # (1/sigma) * (1/tau) 
        gp.set_parameter_vector(params)
        log_a  , log_c =  params
        log_prior = - (log_a / 2.0) + log_c
        return -gp.log_likelihood(y, quiet=True) - log_prior

    if prior is 'Jeff2' : # (1/sigma_hat) * (1/tau) - the one used by Kozlowski , 
        # as well as Chelsea... 
        gp.set_parameter_vector(params)
        log_a  , log_c =  params
        log_prior  = 0.5* (-np.log(2.0) - log_a + log_c  )
        return -gp.log_likelihood(y, quiet=True)  - log_prior

In [ ]:
# initialize the Celerite kernel : 
# same for all light curves ... 
# doesn't matter what it is initialized with 
kernel = terms.RealTerm(log_a = 2 * np.log(0.2) , 
                        log_c = np.log(1.0/100))
SF_inf = 0.2 # mag 
t_exp = 8 * 365.0 # in days 
rho_min, rho_max,  n_rho = 0.001, 15, 100
rho_grid = np.logspace(np.log10(rho_min), np.log10(rho_max), n_rho)



In [1]:
pwd

'/Users/chris/GradResearch/Paper2_SDSS_PTF_PS1/code'

In [ ]:
simulation = 'SDSS'
if simulation is 'SDSS' : 
    r = 17 # mag 
    variance = 0.013**2.0 + np.exp(2 * (r-23.36))
    t = np.loadtxt('t_SDSS.txt')
if simulation is 'OGLE' : 
    I = 18 # mag 
    variance = 0.004**2.0 + np.exp(1.63 * (I - 22.55))    
    t =  np.loadtxt('t_OGLE.txt')
        
#print(simulation,' noise stdev ', np.sqrt(variance))

# set the light curve directory 
outDir = '../data_products/Simulated_DRW_Kozlowski/'+simulation+'/'

# set the output directory 
resDir  = '../data_products/Simulated_DRW_Kozlowski/'+simulation+'/thursday_work/'
        
        
def fit_lightcurve(t,y,yerr,kernel,chain_name, results_name, SF_inf, tau_in,
                   prior='Jeff2',
                  sig_lims =[0.02, 0.7], tau_lims = [1,40000],
                  verbose=True):
            
    results = {}
    sigma_in = SF_inf / np.sqrt(2)
    results['sigma_in'] = sigma_in
    results['tau_in'] = tau_in
    
    if verbose:
        print('sigma_in=' ,sigma_in, 'tau_in=', tau_in)
        
    #log_a, log_c =  2 * np.log(sigma_in),np.log(1.0/tau_in) 
    #truths = [log_a, log_c]
    
    # Find MAP solution with Celerite 
    # call the model  with a chosen kernel instance 
    gp = celerite.GP(kernel, mean=np.mean(y))
    gp.compute(t, yerr)

    # set initial params 
    initial_params = gp.get_parameter_vector()

    # boundaries for the MAP estimate 
    logc_bounds= (np.log(1/max(tau_lims)), 
                  np.log(1/min(tau_lims)) )
    loga_bounds = (2*np.log(min(sig_lims)), 
                   2*np.log(max(sig_lims)))
    bounds = [loga_bounds, logc_bounds]
            
    # calculate 
    gp = celerite.GP(kernel, mean=np.mean(y))
    gp.compute(t, yerr)

     # set initial params 
    initial_params = gp.get_parameter_vector()
             
    # wrap the neg_log_posterior for a chosen prior 
    def neg_log_like(params,y,gp):
        return neg_log_posterior(params,y,gp,prior)

    # find MAP solution 
    r = minimize(neg_log_like, initial_params, 
             method="L-BFGS-B", bounds=bounds, args=(y, gp))
    gp.set_parameter_vector(r.x)
    res = gp.get_parameter_dict()

    tau_fit = np.exp(-res['kernel:log_c'])
    sigma_fit = np.exp(res['kernel:log_a']/2)
    results['tau_MAP'] = tau_fit
    results['sigma_MAP'] = sigma_fit
    
    if verbose : 
        print('MAP ')
        print('sigma_fit', sigma_fit,'tau_fit', tau_fit)
              
            
    # evaluate logP
    Ngrid = 60 
    sigma_grid = np.linspace(sig_lims[0], sig_lims[1],Ngrid )
    tau_grid  = np.linspace(tau_lims[0], tau_lims[1], Ngrid)
    logP = evaluate_logP(sigma_grid, tau_grid,y,gp)
    results['logP'] = logP
    results['sigma_grid'] = sigma_grid
    results['tau_grid'] = tau_grid
    
    ######
    ######  MCMC 
    ###### 

    # wrap the neg_log_posterior for this 
    def log_probability(params,y,gp,prior):
        return -neg_log_posterior(params,y,gp,prior)

    # set the initial chain position on the MAP value 
    initial = np.array(r.x)
    ndim, nwalkers = len(initial), 32

    sampler = emcee.EnsembleSampler(nwalkers, ndim,  log_probability,
                                   args=(y,gp,prior))

    print("Running burn-in...")
    p0 = initial + 1e-8 * np.random.randn(nwalkers, ndim)
    p0, lp, _ = sampler.run_mcmc(p0, 500)

    print("Running production...")
    sampler.reset()
    sampler.run_mcmc(p0, 2000);
    print("Done")

    # Store the chains 
    samples = sampler.flatchain
    np.savetxt(resDir+chain_name,samples )
    
    # Store the median and 16th, 84th percentiles 
    inds = np.array([0,1])
    log_a,log_c = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples[:, inds]
                                                , [16, 50, 84],
                                                axis=0)))

    tau_MCMC_mode, tau_hi, tau_lo = np.exp(-np.asarray(log_c))
    sigma_MCMC_mode ,sigma_hi, sigma_lo= np.exp(np.asarray(log_a)/2)
    
    results['tau_MCMC'] = tau_MCMC_mode
    results['sigma_MCMC'] = sigma_MCMC_mode
    results['tau_PM'] = [tau_lo, tau_hi]
    results['sigma_PM'] = [sigma_lo, sigma_hi]
    
    if verbose : 
        print('the MCMC result for tau is ', tau_MCMC_mode, '+', 
              tau_hi, '-', tau_lo)
        print('the MCMC result for sigma is ', sigma_MCMC_mode, '+', 
              sigma_hi, '-', sigma_lo)
    
    # Save all the results 
    np.save(resDir  + results_name, results)
    if verbose:
        print(' Saved logP (and MAP) dic as %s'%results_name)
        
        

In [ ]:
def evaluate_logP(sigma_grid, tau_grid, y,gp, prior='None'):
    
    log_a_grid = 2 * np.log(sigma_grid)
    log_c_grid = np.log(1/tau_grid)

    # loop over the likelihood space .... 
    logPosterior = np.zeros([len(sigma_grid),len(tau_grid)], 
                            dtype=float)
    for k in range(len(log_a_grid)):
        for l in range(len(log_c_grid)):
            params = [log_a_grid[k],log_c_grid[l]]    
            logPosterior[k,l] = -neg_log_posterior(params,y,gp, prior)
    return logPosterior



In [ ]:


# read in the light curves
for i in range(54,55)#len(rho_grid)): 
    rho_in = rho_grid[i]
    tau_in = rho_in * t_exp
    if i % 10 == 0 : 
        print(i)
    # for each rho, read in  N light curves 
    for j in range(74,100):
        fname = 'DRW_rho-' + str(i).zfill(3)+'_'+str(j).zfill(3)+'.txt'
        y = np.loadtxt(outDir+fname)
        # make a random draw from a Gaussian distribution
        # centered on 0,
        # with variance set by the equation from Kozlowski+2017
        # variance is different for OGLE or  SDSS 
        noise = np.random.normal(loc=0,scale=np.sqrt(variance),size=len(t))
        y += noise +10 # eq.2 Kozlowski+2017

        # the uncertainty on each measurement : 
        # I set it to sigma_SDSS = 0.0248, or sigma_OGLE = 0.0131,
        # homoscedastic, i.e. same errors for all points 
        yerr = np.ones_like(t)* np.sqrt(variance)
        chain_name = fname[:-4]+'_chain.txt'
        results_name = fname[:-4]+'_logP.npy'
        
        # call the MAP, MCMC fitting code 
        fit_lightcurve(t,y,yerr,kernel,chain_name, results_name, SF_inf, tau_in,
                       prior='Jeff2',
                      sig_lims =[0.02, 0.7], tau_lims = [1,40000],
                      verbose=True)
